In [151]:
from torch import optim
from utils import *
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
from TorchCRF import CRF
from typing import List, Tuple, Optional
import math


# import evaluation metrics 
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

torch.manual_seed(42)

In [152]:
if torch.cuda.is_available():
    device = torch.device('cuda')

device

device(type='cuda')

In [153]:
# class ConstrainedCRF(CRF):
#     def __init__(self, num_tags):
#         super(ConstrainedCRF, self).__init__(num_tags)

#     def decode(self, emissions: torch.Tensor, constraints: List[Tuple[torch.IntTensor, torch.IntTensor]], mask: Optional[torch.ByteTensor] = None) -> List[List[int]]:
#         """Find the most likely tag sequence using Viterbi algorithm.

#         Args:
#             emissions (`~torch.Tensor`): Emission score tensor of size
#                 ``(seq_length, batch_size, num_tags)`` if ``batch_first`` is ``False``,
#                 ``(batch_size, seq_length, num_tags)`` otherwise.
#             mask (`~torch.ByteTensor`): Mask tensor of size ``(seq_length, batch_size)``
#                 if ``batch_first`` is ``False``, ``(batch_size, seq_length)`` otherwise.

#         Returns:
#             List of list containing the best tag sequence for each batch.
#         """
#         self._validate(emissions, mask=mask)
#         if mask is None:
#             mask = emissions.new_ones(emissions.shape[:2], dtype=torch.uint8)

#         if self.batch_first:
#             emissions = emissions.transpose(0, 1)
#             mask = mask.transpose(0, 1)

#         return self._viterbi_decode(emissions, mask, constraints)

#     def _viterbi_decode(self, emissions: torch.FloatTensor, mask: torch.ByteTensor, constraints: List[Tuple[torch.IntTensor, torch.IntTensor]]) -> List[List[int]]:
#             print("Viterbi Decode")
#             '''
#                 override the viterbi decode function to include the constraints
#             '''
#             assert emissions.dim() == 3 and mask.dim() == 2 # (sequence_length, batch_size, num_tags)
#             assert emissions.shape[:2] == mask.shape # (sequence_length, batch_size)
#             assert emissions.size(2) == self.num_tags
#             assert mask[0].all()

#             sequence_length, batch_size = mask.shape

#             constrained_transitions = self.transitions.clone()
#             constrainted_satrt_transitions = self.start_transitions.clone()
#             for constraint in constraints:
#                 if constraint[0] == -1:
#                     constrainted_satrt_transitions[constraint[1]] = INF
#                 else:
#                     constrained_transitions[constraint[0], constraint[1]] = INF



#             # start transition and first emission
#             # tensor of size (batch_size, num_tags)
#             # constraints for start transition
#             # score = self.start_transitions + emissions[0] 
#             score = constrainted_satrt_transitions + emissions[0] 

#             backpointers = []

#             for i in range(1, sequence_length):
#                 # Broadcast viterbi score for every possible next tag
#                 # shape: (batch_size, num_tags, 1)
#                 broadcast_score = score.unsqueeze(2)
                
                    
#                 # Broadcast emission score for every possible current tag
#                 # shape: (batch_size, 1, num_tags)
#                 broadcast_emission = emissions[i].unsqueeze(1)

#                 # Compute the score tensor of size (batch_size, num_tags, num_tags) where
#                 # for each sample, entry at row i and column j stores the score of the best
#                 # tag sequence so far that ends with transitioning from tag i to tag j and emitting
#                 # shape: (batch_size, num_tags, num_tags)
#                 # next_score = broadcast_score + self.transitions + broadcast_emission
#                 next_score = broadcast_score + constrained_transitions + broadcast_emission

#                 # print("Next Score Shape:", next_score[0].shape)

#                 # Apply the constraints
#                 # for constraint in constraints:
#                 #     # print("Constraint:", constraint)
#                 #     next_score[0][constraint[0], constraint[1]] = INF

                
#                 # Find the maximum score over all possible current tag
#                 # shape: (batch_size, num_tags)
#                 next_score, indices = next_score.max(dim=1)

#                 # Set score to the next score if this timestep is valid (mask == 1)
#                 # and save the index that produces the next score
#                 # shape: (batch_size, num_tags)
#                 score = torch.where(mask[i].unsqueeze(1), next_score, score)
#                 backpointers.append(indices)
            
            
#             # End transition score
#             # shape: (batch_size, num_tags)
#             score += self.end_transitions


#             # Now, compute the best path for each sample

#             # shape: (batch_size,)
#             seq_ends = mask.long().sum(dim=0) - 1
#             best_tags_list = []

#             for idx in range(batch_size):
#                 # Find the tag which maximizes the score at the last timestep; this is our best tag
#                 # for the last timestep
#                 _, best_last_tag = score[idx].max(dim=0)
#                 best_tags = [best_last_tag.item()]

#                 # We trace back where the best last tag comes from, append that to our best tag
#                 # sequence, and trace it back again, and so on
#                 for hist in reversed(backpointers[:seq_ends[idx]]):
#                     best_last_tag = hist[idx][best_tags[-1]]
#                     best_tags.append(best_last_tag.item())

#                 # Reverse the order because we start from the last timestep
#                 best_tags.reverse()
#                 best_tags_list.append(best_tags)

#             return best_tags_list



# class BiLSTMCRF(nn.Module):
#     def __init__(self, vocab_size, word_embedding_dim, intent_embedding_dim, hidden_dim, output_dim, number_of_intents, index_to_tag):
        
#         super(BiLSTMCRF, self).__init__()
#         # hyperparameters
#         self.word_embedding_dim = word_embedding_dim
#         self.inten_embedding_dim = intent_embedding_dim
#         self.hidden_dim = hidden_dim
#         self.vocab_size = vocab_size
#         self.output_dim = output_dim
#         self.index_to_tag = index_to_tag

#         # model layers
#         self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
#         self.intent_embedding = nn.Embedding(number_of_intents, intent_embedding_dim)
#         self.lstm = nn.LSTM(word_embedding_dim + intent_embedding_dim, hidden_dim // 2, bidirectional=True, dropout=0.2)
#         self.hidden_to_tag = nn.Linear(hidden_dim, output_dim)
#         self.crf = ConstrainedCRF(output_dim)

#     def __create_constraints(self, mask, intent):
#         constraints = []
#         one_indices = torch.where(mask == 1)[0] # tags that are for the intent
#         zero_indices = torch.where(mask == 0)[0] # tags that are not for the intent
        
#         print(one_indices)
#         print(zero_indices)

#         for i in one_indices:
#             # add constraints for the tags that are related to the intent
#             # the transition from an intent tag to a non-intent tag is not allowed
#             for j in zero_indices:
#                 constraints.append((i, j))
#                 constraints.append((j, i))
            
#             # constraints that the tag if O can't be followed by I
#             # but I can be followed by O 
#             if self.index_to_tag[i.item()] == 'O':
#                 for j in one_indices:
#                     if self.index_to_tag[j.item()][0] == 'I':
#                         constraints.append((i, j))
        
#             # add constraints that if the tag begins with B then the next tag can't begin with B of the same 
#             # if tag begins with B then it can't be followed by B of the same entity
#             # or I of a different entity
#             # it can be followed by I of same entity, O or B of a different entity
#             # add constraints that if the tag begins with B then the next tag cant begin with I of a different entity
#             if self.index_to_tag[i.item()][0] == 'B':
#                 constraints.append((i, i))
#                 for j in one_indices:
#                     if self.index_to_tag[j.item()][0] == 'I' and self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
#                         constraints.append((i, j))
                        
#             # if self.index_to_tag[i.item()][0] == 'B':
#             #     for j in one_indices:
#             #         if self.index_to_tag[j.item()][0] == 'I' and self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
#             #             constraints.append((i, j))

#             # create start constraints
#             # if the tag starts with I then it can't be the first tag
#             if self.index_to_tag[i.item()][0] == 'I':
#                 constraints.append((-1, i))
#                 for j in one_indices:
#                     if self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
#                         constraints.append((i, j))
#                         constraints.append((j, i))
                        
#             # add constraint the if a tag starts with I then the previous tag should be the same entity
#             # the for example I-CONDITION cant be followed by I-VAL and vice versa
#             # if self.index_to_tag[i.item()][0] == 'I':
#             #     for j in one_indices:
#             #         if self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
#             #             constraints.append((i, j))
#             #             constraints.append((j, i))
                        

#         # add constraints for the tags that are not related to the intent at all
#         for i in zero_indices:
#             for j in zero_indices:
#                 constraints.append((i, j))


#         return constraints

#     def __init_hidden(self):
#         return (torch.randn(2, 1, self.hidden_dim // 2), torch.randn(2, 1, self.hidden_dim // 2)) # initialize hidden state

#     def __get_lstm_features(self, sentence, intent):
#         self.hidden = self.__init_hidden()
#         word_embeddings = self.word_embedding(sentence).view(len(sentence), 1, -1)
#         # print("Word Embedding Shape:", word_embeddings.shape) # (len(sentence), batch_size, embedding_dimension)
#         intent_embeddings = self.intent_embedding(intent).view(1, 1, -1).repeat(len(sentence), 1, 1)
#         # print("Intent Embedding Shape:", intent_embeddings.shape) # (len(sentence), batch_size, embedding_dimension)
#         embeddings = torch.cat((word_embeddings, intent_embeddings), dim=2) 
#         # print("Embedding Shape:", embeddings.shape) # (len(sentence), batch_size, embedding_dimension * 2)
#         lstm_out, self.hidden = self.lstm(embeddings, self.hidden)
#         # print("LSTM Out Shape:", lstm_out.shape) # (len(sentence), batch_size, hidden_dimension)
#         lstm_features = self.hidden_to_tag(lstm_out)
#         # print("LSTM Features Shape:", lstm_features.shape) # (len(sentence), batch_size, output_dimension (number of target entities)) -> number of entities 3andena 30 
#         return lstm_features

#     def neg_log_likelihood(self, sentence, tags, intent, mask):
#         # print("Intent Shape:", intent.shape)
#         emissions = self.__get_lstm_features(sentence, intent) # (sentence_length, batch_size, number of tags) # emissions where each word in the sequence coressponds to probability for tag
#         # print("NEG LIKE FEATURES:", features.shape)
#         # print(features)
#         # mask = mask.view(1, -1)
#         # print("Mask Shape:", mask.shape)
#         # repeated_mask = mask.repeat(features.shape[0], 1, 1).contiguous()
#         # print(repeated_mask.shape)
#         # print(repeated_mask)
#         # features = features * mask
#         # print("Intent:", intent[0])
#         # print("Mask:", mask)
#         # print("Features:", features)
#         # print("Features Shape:", features.shape)
#         # mask = mask.view(-1, 1)
#         # print("Emissions[0] before:", emissions[0])
#         # print("Mask:", mask)
#         intent_mask = torch.where(mask == 0, torch.tensor([INF]), torch.tensor([0.0]))
#         # print("Log Mask:", intent_mask)
#         # intent_mask = torch.tensor([1 if i not in torch.where(mask == 0)[0] else INF for i, mask in enumerate(mask)], dtype=torch.long)
#         # intent_mask = intent_mask.view(1, 1, -1)
#         # print("Intent Mask:", intent_mask)
#         emissions = emissions + intent_mask
#         # print("Mask Shape:", intent_mask.shape)
#         # print("Emissions Shape", emissions.shape)
#         # print("Emissions[0] after:", emissions)
#         tags = tags.view(-1, 1)
#         loss = -self.crf(emissions, tags)
#         # print("Loss:", loss)
#         return loss

#     def forward(self, sentence, intent, mask):
#         emissions = self.__get_lstm_features(sentence, intent)
#         # print(lstm_features)
#         # print("LSTM Features", lstm_features)
#         # print("Forward LSTM features:", lstm_features.shape)
#         # repeated_mask = mask.repeat(lstm_features.shape[0], 1, 1).contiguous()
#         # test_mask = torch.ones(len(sentence), 1, self.output_dim)
#         # print("Repeated Mask Shape:", repeated_mask.shape)
#         # print("Mask Shape:", mask.shape)
#         # lstm_features = lstm_features * mask
#         # print("Forward LSTM features:", lstm_features.shape)
#         # print(self.crf.transitions)
#         print(mask)
#         intent_mask = torch.where(mask == 0, torch.tensor([INF]), torch.tensor([0.0]))
#         emissions = emissions + intent_mask
#         constraints = self.__create_constraints(mask, intent)
#         tag_sequence = self.crf.decode(emissions, constraints)
#         # print("Tag Sequence:", tag_sequence)
#         return tag_sequence
    

In [154]:
INF = -100

class ConstrainedCRF(CRF):
    def __init__(self, num_tags):
        super(ConstrainedCRF, self).__init__(num_tags)

    def decode(self, emissions: torch.Tensor, constraints: List[Tuple[torch.IntTensor, torch.IntTensor]], mask: Optional[torch.ByteTensor] = None) -> List[List[int]]:
        """Find the most likely tag sequence using Viterbi algorithm.

        Args:
            emissions:  
            mask:  
            constraints: List of tuples containing the constraints for the CRF.

        Returns:
            List of list containing the best tag sequence for each batch.
        """
        self._validate(emissions, mask=mask)
        if mask is None:
            mask = emissions.new_ones(emissions.shape[:2], dtype=torch.uint8)

        if self.batch_first:
            emissions = emissions.transpose(0, 1)
            mask = mask.transpose(0, 1)

        return self._viterbi_decode(emissions, mask, constraints)

    def _viterbi_decode(self, emissions: torch.FloatTensor, mask: torch.ByteTensor, constraints: List[Tuple[torch.IntTensor, torch.IntTensor]]) -> List[List[int]]:
        print("Viterbi Decode")
        '''
            override the viterbi decode function to include the constraints
        '''
        assert emissions.dim() == 3 and mask.dim() == 2  # (sequence_length, batch_size, num_tags)
        assert emissions.shape[:2] == mask.shape
        assert emissions.size(2) == self.num_tags
        assert mask[0].all()

        # get the sequence length and batch size
        sequence_length, batch_size = mask.shape

        # first we apply the constraints to the transitions
        constrained_transitions = self.transitions.clone()
        constrainted_satrt_transitions = self.start_transitions.clone()
        for constraint in constraints:
            if constraint[0] == -1:
                # start transition constraint
                constrainted_satrt_transitions[constraint[1]] = INF
            else:
                constrained_transitions[constraint[0], constraint[1]] = INF

        # start transition and first emission
        # tensor of size (batch_size, num_tags)
        # constraints for start transition
        score = constrainted_satrt_transitions + emissions[0]

        # back pointers to store the tag sequences that give the best score
        # keeps tracks of the best paths up to the current timestep
        backpointers = []

        # we start from 1 since the first timestep is already computed
        # using the start transition and the first emission
        for i in range(1, sequence_length):

            # broadcast the current score for every possible next tag
            # the score contains the score of the best path up to the current timestep
            # shape: (batch_size, num_tags, 1)
            broadcast_score = score.unsqueeze(2)

            # emission[i] is the emission score for the current timestep
            # broadcast the emission score for every possible current tag
            # shape: (batch_size, 1, num_tags)
            broadcast_emission = emissions[i].unsqueeze(1)

            # the addition of the broadcasted score and the broadcasted emission and the trainsition score
            # result int the shape (batch_size, num_tags, num_tags) where each entry (i, j, k)
            # the score for transitioning from tag j to tag k at the current timestep
            # and emitting k
            next_score = broadcast_score + constrained_transitions + broadcast_emission

            # Find the maximum score over all possible current tag
            # shape: (batch_size, num_tags) the maximum score for each possible current tag to update the score
            # indices: (batch_size, num_tags) the index of the tag that gives the maximum score
            next_score, indices = next_score.max(dim=1)

            # update the score and the backpointers
            score = torch.where(mask[i].unsqueeze(1), next_score, score)

            # add the indices to the backpointers
            backpointers.append(indices)

        # End transition score
        # shape: (batch_size, num_tags)
        score += self.end_transitions

        # Now that we have the final score, we can trace back the best path

        # shape: (batch_size,) containing the length of each sequence
        seq_ends = mask.long().sum(dim=0) - 1

        # shape: (batch_size, num_tags) containing the score of the best tag for each sequence
        # in our case will only have one since our batch size is 1
        best_tags_list = []

        # loop on tags for each sequence in the batch (again 1)
        for idx in range(batch_size):

            # find the best tag for the last timestep
            _, best_last_tag = score[idx].max(dim=0)
            best_tags = [best_last_tag.item()]

            # backtrace the best tag for each timestep
            # we move from the last timestep to the first
            for hist in reversed(backpointers[:seq_ends[idx]]):
                best_last_tag = hist[idx][best_tags[-1]]
                best_tags.append(best_last_tag.item())

            # reverse the order
            best_tags.reverse()

            # again we only have one sequence in the batch
            best_tags_list.append(best_tags)

        return best_tags_list


class NERModel(nn.Module):
    def __init__(self, vocab_size, word_embedding_dim, intent_embedding_dim, hidden_dim, output_dim, number_of_intents, index_to_tag):

        super(NERModel, self).__init__()
        # hyperparameters
        self.word_embedding_dim = word_embedding_dim
        self.inten_embedding_dim = intent_embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.index_to_tag = index_to_tag

        # model layers
        self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
        self.intent_embedding = nn.Embedding(number_of_intents, intent_embedding_dim)
        self.lstm = nn.LSTM(word_embedding_dim + intent_embedding_dim,hidden_dim // 2, bidirectional=True, dropout=0.4)
        self.hidden_to_tag = nn.Linear(hidden_dim, output_dim)
        self.crf = ConstrainedCRF(output_dim)

    def __create_constraints(self, mask, intent):
        constraints = []
        one_indices = torch.where(mask == 1)[0]  # tags that are for the intent
        # tags that are not for the intent
        zero_indices = torch.where(mask == 0)[0]

        # print(one_indices)
        # print(zero_indices)

        for i in one_indices:
            # add constraints for the tags that are related to the intent
            # the transition from an intent tag to a non-intent tag is not allowed
            for j in zero_indices:
                constraints.append((i, j))
                constraints.append((j, i))

            # constraints that the tag if O can't be followed by I
            # but I can be followed by O
            # if self.index_to_tag[i.item()] == 'O':
            #     for j in one_indices:
            #         if self.index_to_tag[j.item()][0] == 'I':
            #             constraints.append((i, j))

            # add constraints that if the tag begins with B then the next tag can't begin with B of the same
            # if tag begins with B then it can't be followed by B of the same entity
            # or I of a different entity
            # it can be followed by I of same entity, O or B of a different entity
            # add constraints that if the tag begins with B then the next tag cant begin with I of a different entity
            # if self.index_to_tag[i.item()][0] == 'B':
            #     constraints.append((i, i))
            #     for j in one_indices:
            #         if self.index_to_tag[j.item()][0] == 'I' and self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
            #             constraints.append((i, j))

            # create start constraints
            # if the tag starts with I then it can't be the first tag
            if self.index_to_tag[i.item()][0] == 'I':
                constraints.append((-1, i))
                # for j in one_indices:
                    # if self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
                    #     constraints.append((i, j))
                    #     constraints.append((j, i))

        # add constraints for the tags that are not related to the intent at all
        for i in zero_indices:
            for j in zero_indices:
                constraints.append((i, j))
                constraints.append((j, i))

        return constraints

    def __init_hidden(self):
        '''
            Initialize the hidden state of the LSTM
        '''
        return (torch.randn(2, 1, self.hidden_dim // 2), torch.randn(2, 1, self.hidden_dim // 2))

    def __get_emissions(self, sentence, intent):
        # initialize the hidden state for the lstm
        # the hidden state is initialized for each input sequence
        # to prevent cascading information from on input sequence to the other
        # as the sentences are independent
        # this also helps prevent overfitting of the model
        self.hidden = self.__init_hidden()

        # generate the word embedding for each word in the input sequence
        # captures the semantic information of the sentence
        word_embeddings = self.word_embedding(sentence).view(len(sentence), 1, -1)

        # generate the intent embedding for the sequence intent
        # to sway the model to choose from certain tags based on the inten
        # by learning semantic information about the intent
        # since there is a single intent for each sentence but we concatenate it with the word embedding
        # we repeat the embedding with the length of the sequence
        intent_embeddings = self.intent_embedding(intent).view(1, 1, -1).repeat(len(sentence), 1, 1)

        # the final embeddings are concatenated to be passed to the lstm layer
        embeddings = torch.cat((word_embeddings, intent_embeddings), dim=2)
        # 
        lstm_out, self.hidden = self.lstm(embeddings, self.hidden)

        lstm_features = self.hidden_to_tag(lstm_out)

        return lstm_features

    def negative_log_likelihood(self, sentence, tags, intent, mask):

        emissions = self.__get_emissions(sentence, intent)

        intent_mask = torch.where(mask == 0, torch.tensor([INF]), torch.tensor([0.0]))

        emissions = emissions + intent_mask

        tags = tags.view(-1, 1)
        loss = -self.crf(emissions, tags)
        return loss

    def forward(self, sentence, intent, mask):
        # get the emission scores from the BiLSTM
        # the emission score represent the score that a word is a certain tag
        # the score is calculated based on the word embedding and the intent embedding
        emissions = self.__get_emissions(sentence, intent)

        # since we are using the negative log likelihood loss
        # map the 0s to -inf and the 1s to 0
        # as the scores could be are negative
        intent_mask = torch.where(
            mask == 0, torch.tensor([INF]), torch.tensor([0.0]))

        # adjust the emissions to account for the intent mask
        # the emission score for the tags that are not related to the intent are set to -inf
        emissions = emissions + intent_mask

        # baseed on the mask and intent create the constraints
        # the constraints are used to guide the CRF to select the best tag sequence
        # by adjusting the transition scores from one tag to another
        constraints = self.__create_constraints(mask, intent)

        # decode the best tag sequence using the CRF
        # using the emissions and the constraints
        tag_sequence = self.crf.decode(emissions, constraints)

        return tag_sequence


### **HELPER FUNCTIONS**

In [155]:
# constants
WORD_EMBEDDING_DIM = 50
INTENT_EMBEDIING_DIM = 50
HIDDEN_DIM = 64

In [156]:
def read_dataset():
    data = pd.read_csv('../ner_dataset/ner_dataset.csv', encoding='latin1')

    # remove white spaces from column names
    data.columns = data.columns.str.strip()

    print(data.columns)
    # Group by 'Sentence #' and aggregate
    grouped_data = data.groupby('Sentence #').agg({
        'Word': lambda x: ''.join(x),  # Join words into a single sentence
        'Tag': lambda x: list(x.str.strip()),       # Collect tags into a list
        'Intent': lambda x: list(x.str.strip().str.replace('_', ' '))     # Collect intents into a list
    }).reset_index()  # Reset index to make 'Sentence #' a regular column

    return data, grouped_data


def prepare_data(dataframe):
    dataset = []
    # a dictionary with the format 
    # intent: [[sentences], [tags]]
    other_dataset = {} # this will be spearated by intent to split the data to train and validation for each intent
    i = 0
    for _, row in dataframe.iterrows():
        sentence = row['Word'][1:]
        tags = row['Tag']
        intents = row['Intent']
        dataset.append((sentence, tags, intents[0]))

        if intents[0] not in other_dataset:
            other_dataset[intents[0]] = [[sentence], [tags]]
        else:
            other_dataset[intents[0]][0].append(sentence)
            other_dataset[intents[0]][1].append(tags)

    return dataset, other_dataset

In [157]:
# Format data
data, goruped_data = read_dataset()
print(data["Intent"].unique())

goruped_data.head()

Index(['Sentence #', 'Word', 'Tag', 'Intent'], dtype='object')
[' array operation' ' assertion' ' assignment operation'
 ' bitwise operation' ' casting' ' class declaration' ' comment'
 ' conditional operation' ' constant declaration' ' file system'
 ' for loop' ' function declaration' ' git operation' ' ide operation'
 ' input' ' interactive commands' ' libraries' ' mathematical operation'
 ' membership operation' ' output' ' variable declaration' ' while loop']


,Sentence #,Word,Tag,Intent
0,0,append the value 12345.6789 to the list packe...,"[B-OPERATION, O, O, B-ELEMENT, O, O, B-ARRAY, ...","[array operation, array operation, array opera..."
1,1,push the value false to the list inventories,"[B-OPERATION, O, O, B-ELEMENT, O, O, O, B-ARRAY]","[array operation, array operation, array opera..."
2,2,append the value -456 to the array player scores,"[B-OPERATION, O, O, B-ELEMENT, O, O, O, B-ARRA...","[array operation, array operation, array opera..."
3,3,push the value 3.14159 to the list links,"[B-OPERATION, O, O, B-ELEMENT, O, O, O, B-ARRAY]","[array operation, array operation, array opera..."
4,4,include the value Back End Development to the...,"[B-OPERATION, O, O, B-ELEMENT, I-ELEMENT, I-EL...","[array operation, array operation, array opera..."


In [158]:
# training data = (sentence, tags, intent)
training_data, training_validation_data = prepare_data(goruped_data)

print(training_data[0])
print(len(training_data))

print(len(training_validation_data))
print(training_validation_data['array operation'])

('append the value 12345.6789 to the list packet list', ['B-OPERATION', 'O', 'O', 'B-ELEMENT', 'O', 'O', 'B-ARRAY', 'I-ARRAY', 'O'], 'array operation')
2868
22
[['append the value 12345.6789 to the list packet list', 'push the value false to the list inventories', 'append the value -456 to the array player scores', 'push the value 3.14159 to the list links', 'include the value Back End Development to the array routes', 'add the value this is a test to the list window components', 'push the value data structures to the list wishlist items', 'insert the value -7654321 to the list test data', 'add the value continuous integration to the array label array', 'place the value 789 to the array templates', 'add the value Full Stack Developer to the list quest items', 'insert the value functional programming to the array label array', 'insert -141414 to the array icon set', 'put -80808 to the array events', 'append 444444 to the array checkboxes', 'insert REGRESSION MODELS to the array categori

In [159]:
# split the data to train and validation based on the intent
other_train_data = []
other_validation_data = []

for intent in training_validation_data:
    sentences = training_validation_data[intent][0]
    tags = training_validation_data[intent][1]
    split_index = math.floor(len(sentences) * 0.8)
    other_train_data.extend([(sentences[i], tags[i], intent) for i in range(split_index)])
    other_validation_data.extend([(sentences[i], tags[i], intent) for i in range(split_index, len(sentences))])

print(len(other_train_data))
print(len(other_validation_data))

assert len(training_data) == len(other_train_data) + len(other_validation_data)


2291
577


### **PREPARE DATA**

In [160]:
# convert intent to index
with open('../ner_dataset/annotations/annotations.json', 'r') as f:
    data = json.load(f)

annotations = data['annotations']

intents = list(annotations.keys())

print(intents)
print(len(intents))

intent_to_index = {intent: index for index, intent in enumerate(intents)}

# intent_to_index["<UNK>"] = len(intent_to_index)

['array operation', 'assertion', 'assignment operation', 'bitwise operation', 'casting', 'class declaration', 'comment', 'conditional operation', 'constant declaration', 'file system', 'for loop', 'function declaration', 'git operation', 'ide operation', 'input', 'interactive commands', 'libraries', 'mathematical operation', 'membership operation', 'output', 'variable declaration', 'while loop']
22


In [161]:
# convert data to indices
word_to_index = {}
tag_to_index = {}
for sentence, tags, intents in other_train_data:
    for word in sentence.split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
            
    for tag in tags:
        if tag not in tag_to_index:
            tag_to_index[tag] = len(tag_to_index)

# add the 'UNK' token
word_to_index['<UNK>'] = len(word_to_index)

tag_to_index['<UNK>'] = len(tag_to_index)

tag_to_index

{'B-OPERATION': 0,
 'O': 1,
 'B-ELEMENT': 2,
 'B-ARRAY': 3,
 'I-ARRAY': 4,
 'I-ELEMENT': 5,
 'I-OPERATION': 6,
 'B-LHS': 7,
 'B-CONDITION': 8,
 'I-CONDITION': 9,
 'B-RHS': 10,
 'I-RHS': 11,
 'I-LHS': 12,
 'B-OPERATOR': 13,
 'B-OPERAND': 14,
 'I-OPERAND': 15,
 'I-OPERATOR': 16,
 'B-VAR': 17,
 'I-VAR': 18,
 'B-TYPE': 19,
 'B-CLASS': 20,
 'I-CLASS': 21,
 'I-COMMENT': 22,
 'B-COMMENT': 23,
 'B-LOG': 24,
 'B-VAL': 25,
 'I-VAL': 26,
 'B-ACTION': 27,
 'I-ACTION': 28,
 'B-DIR': 29,
 'B-START': 30,
 'B-END': 31,
 'B-STEP': 32,
 'B-COLLECTION': 33,
 'I-COLLECTION': 34,
 'B-FUNC': 35,
 'I-FUNC': 36,
 'B-PARAM': 37,
 'I-PARAM': 38,
 'B-MESSAGE': 39,
 'I-MESSAGE': 40,
 'B-LINE': 41,
 'B-FILE': 42,
 'B-LIB_NAME': 43,
 '<UNK>': 44}

In [162]:
vocab_size = len(word_to_index)

print("Vocabulary Size: ", vocab_size)

number_of_tags = len(tag_to_index)

print("Number of Tags: ", number_of_tags)

number_of_intents = len(intent_to_index)

print("Number of Intents: ", number_of_intents)

Vocabulary Size:  1034
Number of Tags:  45
Number of Intents:  22


In [163]:
tag_to_index
index_to_tag = {index: tag for tag, index in tag_to_index.items()}
index_to_tag

{0: 'B-OPERATION',
 1: 'O',
 2: 'B-ELEMENT',
 3: 'B-ARRAY',
 4: 'I-ARRAY',
 5: 'I-ELEMENT',
 6: 'I-OPERATION',
 7: 'B-LHS',
 8: 'B-CONDITION',
 9: 'I-CONDITION',
 10: 'B-RHS',
 11: 'I-RHS',
 12: 'I-LHS',
 13: 'B-OPERATOR',
 14: 'B-OPERAND',
 15: 'I-OPERAND',
 16: 'I-OPERATOR',
 17: 'B-VAR',
 18: 'I-VAR',
 19: 'B-TYPE',
 20: 'B-CLASS',
 21: 'I-CLASS',
 22: 'I-COMMENT',
 23: 'B-COMMENT',
 24: 'B-LOG',
 25: 'B-VAL',
 26: 'I-VAL',
 27: 'B-ACTION',
 28: 'I-ACTION',
 29: 'B-DIR',
 30: 'B-START',
 31: 'B-END',
 32: 'B-STEP',
 33: 'B-COLLECTION',
 34: 'I-COLLECTION',
 35: 'B-FUNC',
 36: 'I-FUNC',
 37: 'B-PARAM',
 38: 'I-PARAM',
 39: 'B-MESSAGE',
 40: 'I-MESSAGE',
 41: 'B-LINE',
 42: 'B-FILE',
 43: 'B-LIB_NAME',
 44: '<UNK>'}

In [164]:
# Assertions
i = 0
for sentence, tags, intents in training_data:
    if len(sentence.split()) != len(tags):
        print(f"Example {i}: Sentence Length: {len(sentence.split())}, Tags Length: {len(tags)}")
        print("Example:", training_data[i])
        # sentence += chr(random.randint(65, 90))
        # other_train_data[i] = (sentence, tags, intents)
    i +=1

for sentence, tags, intents in training_data:
    assert len(sentence.split()) == len(tags)

In [165]:
# get the tags for each intent
with open('../ner_dataset/intent_to_tags.json', 'w') as intent_to_tags_file:
    intent_to_tags = {}

    for file in os.listdir('../ner_dataset/annotations/final_annotations'):
        with open(f'../ner_dataset/annotations/final_annotations/{file}', 'r') as f:
            data = json.load(f)
            # print(data)
            tags = data['classes']
            tags.append('O')
            # print(tags)
            intent = file.split('.')[0].replace('_', ' ')
            intent_to_tags[intent] = tags
            # print(intent_to_tags[intent])

    json.dump(intent_to_tags, intent_to_tags_file, indent=4)


all_tags = list(tag_to_index.keys())

### **INTENTS HANDLING**

In [166]:
# here we want to filter out the tags that are not relevent to the given intent
def create_mask(intent, all_tags, intent_to_tags):
    intent_tags = intent_to_tags[intent]
    final_tags = []


    # create BI tags for the intent
    for tag in intent_tags:
        # print(tag)
        if tag == 'O': 
            final_tags.append(tag)
            continue
        
        final_tags.append('B-' + tag)
        final_tags.append('I-' + tag)

    mask = [tag in final_tags for tag in all_tags]
        
    mask = torch.tensor(mask, dtype=torch.long)
    return mask

mask = create_mask('variable declaration', all_tags, intent_to_tags)
print(mask)
print(len(mask))

tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
45


### **MODEL DEFINITION**

In [167]:
ner_model = NERModel(vocab_size=len(word_to_index), word_embedding_dim=WORD_EMBEDDING_DIM, intent_embedding_dim=INTENT_EMBEDIING_DIM, hidden_dim=HIDDEN_DIM, output_dim=number_of_tags, number_of_intents=number_of_intents, index_to_tag=index_to_tag)
ner_optimizer = optim.SGD(ner_model.parameters(), lr=0.01, weight_decay=1e-4)

c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [168]:
# Check predictions before training
precheck_tag = prepare_sequence(other_train_data[0][1], tag_to_index)
print(precheck_tag)

mask = create_mask(other_train_data[0][2], all_tags, intent_to_tags)

print(mask)

with torch.no_grad():
    precheck_sent = prepare_sequence(other_train_data[0][0].split(), word_to_index)
    intent = other_train_data[0][2]
    # precheck_intent = prepare_sequence(intent[0], intent_to_index)
    precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
    print(ner_model(precheck_sent, precheck_intent, mask))

tensor([0, 1, 1, 2, 1, 1, 3, 4, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Viterbi Decode
[[3, 1, 1, 1, 1, 1, 1, 1, 1]]


In [169]:
final_tags = {}
for intent in intent_to_tags.keys():
    final_tags[intent] = []
    for tag in intent_to_tags[intent]:
        if tag == 'O': 
            final_tags[intent].append(tag)
            continue
        
        final_tags[intent].append('B-' + tag)
        final_tags[intent].append('I-' + tag)   

final_tags

distinct_tags = final_tags.values()

distinct_tags = set([tag for sublist in distinct_tags for tag in sublist])

distinct_tags

print(len(distinct_tags))

tags_to_index = {tag: index for index, tag in enumerate(distinct_tags)}

tags_to_index

53


{'B-ELEMENT': 0,
 'B-START': 1,
 'I-START': 2,
 'B-OPERAND': 3,
 'I-ARRAY': 4,
 'I-ELEMENT': 5,
 'B-END': 6,
 'I-STEP': 7,
 'B-ARRAY': 8,
 'I-TYPE': 9,
 'B-CLASS': 10,
 'B-VAL': 11,
 'I-LHS': 12,
 'B-OPERATOR': 13,
 'I-COLLECTION': 14,
 'B-FILE': 15,
 'B-ACTION': 16,
 'B-STEP': 17,
 'I-CLASS': 18,
 'I-MESSAGE': 19,
 'B-CONDITION': 20,
 'I-RHS': 21,
 'B-MESSAGE': 22,
 'I-DIR': 23,
 'I-FUNC': 24,
 'I-OPERATOR': 25,
 'O': 26,
 'B-LINE': 27,
 'I-ACTION': 28,
 'B-LOG': 29,
 'B-RHS': 30,
 'I-FILE': 31,
 'I-PARAM': 32,
 'B-COMMENT': 33,
 'I-OPERAND': 34,
 'I-VAR': 35,
 'I-COMMENT': 36,
 'B-LHS': 37,
 'B-VAR': 38,
 'B-OPERATION': 39,
 'I-OPERATION': 40,
 'B-DIR': 41,
 'B-COLLECTION': 42,
 'B-FUNC': 43,
 'I-LINE': 44,
 'I-LOG': 45,
 'I-CONDITION': 46,
 'I-LIB_NAME': 47,
 'I-END': 48,
 'B-PARAM': 49,
 'B-TYPE': 50,
 'I-VAL': 51,
 'B-LIB_NAME': 52}

In [170]:
intent_to_index

{'array operation': 0,
 'assertion': 1,
 'assignment operation': 2,
 'bitwise operation': 3,
 'casting': 4,
 'class declaration': 5,
 'comment': 6,
 'conditional operation': 7,
 'constant declaration': 8,
 'file system': 9,
 'for loop': 10,
 'function declaration': 11,
 'git operation': 12,
 'ide operation': 13,
 'input': 14,
 'interactive commands': 15,
 'libraries': 16,
 'mathematical operation': 17,
 'membership operation': 18,
 'output': 19,
 'variable declaration': 20,
 'while loop': 21}

In [171]:
# split the data into training and testing
# shuffle the data
from sklearn.model_selection import train_test_split


# random.shuffle(training_data)

# split the data
train_data, test_data = train_test_split(training_data, test_size=0.2, random_state=42, shuffle=True)

In [172]:
# training loop
def train(model, optimizer, training_data, epochs=10):
    for epoch in tqdm(range(epochs)):
        total_loss = 0
        for sentence, tags, intent in training_data:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Step 2. Get our inputs ready for the network, that is,
            # turn them into Tensors of word indices.
            intent_mask = create_mask(intent, all_tags, intent_to_tags)

            sentence = prepare_sequence(sentence.split(), word_to_index)
            target_tags = torch.tensor([tag_to_index[t] for t in tags], dtype=torch.long)
            intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)


            # Step 3. Run our forward pass.
            loss = model.negative_log_likelihood(sentence, target_tags, intent, intent_mask)

            total_loss += loss.item()

            # Step 4. Compute the loss, gradients, and update the parameters by
            loss.backward()
            optimizer.step()

        
        print(f"Epoch: {epoch}, Loss: {total_loss / len(training_data)}")


train(ner_model, ner_optimizer, training_data, epochs=100)

  1%|          | 1/100 [00:49<1:21:03, 49.13s/it]

Epoch: 0, Loss: 5.082406729480545


  2%|▏         | 2/100 [01:27<1:10:18, 43.05s/it]

Epoch: 1, Loss: 2.3503517422050946


  3%|▎         | 3/100 [02:13<1:11:16, 44.08s/it]

...

Epoch: 97, Loss: 0.05009749444458774


 99%|█████████▉| 99/100 [58:20<00:29, 29.10s/it]

Epoch: 98, Loss: 0.05041706063258598


100%|██████████| 100/100 [58:44<00:00, 35.25s/it]

Epoch: 99, Loss: 0.05620836280546095


In [173]:
# Check predictions before training
precheck_tag = prepare_sequence(other_train_data[0][1], tag_to_index)
print(precheck_tag)

mask = create_mask(other_train_data[0][2], all_tags, intent_to_tags)

print(mask)

with torch.no_grad():
    precheck_sent = prepare_sequence(other_train_data[0][0].split(), word_to_index)
    intent = other_train_data[0][2]
    # precheck_intent = prepare_sequence(intent[0], intent_to_index)
    precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
    print(ner_model(precheck_sent, precheck_intent, mask))

tensor([0, 1, 1, 2, 1, 1, 3, 4, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Viterbi Decode
[[0, 1, 1, 2, 1, 1, 3, 4, 1]]


In [174]:
def predict(model, test_data):
    predictions = [] 
    with torch.no_grad():
        for sentence, _, intent in test_data:
            precheck_sent = prepare_sequence(sentence.split(), word_to_index)
            # precheck_intent = prepare_sequence(intent, intent_to_index)
            precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
            mask = create_mask(intent, all_tags, intent_to_tags)
            predictions.append(model(precheck_sent, precheck_intent, mask)[0])
    
    return predictions

In [175]:
# validation data
print(len(other_validation_data))
predictions = predict(ner_model, other_validation_data)

577
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterb

In [176]:
labels = []
predicted_tags = []

for index, prediction in enumerate(predictions):
    print("Test Example:", index)
    mapped_tags = [all_tags[tag] for tag in prediction]
    predicted_tags.append(mapped_tags)
    labels.append(other_validation_data[index][1])
    print("Sentence:", other_validation_data[index][0])
    print("Actual tags:", other_validation_data[index][1])
    print("Predicted tags:", mapped_tags)

Test Example: 0
Sentence: reverse the list element list
Actual tags: ['B-OPERATION', 'O', 'B-ARRAY', 'I-ARRAY', 'O']
Predicted tags: ['B-OPERATION', 'O', 'B-ARRAY', 'I-ARRAY', 'O']
Test Example: 1
Sentence: reverse the list responses
Actual tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY']
Predicted tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY']
Test Example: 2
Sentence: reverse the list cookies
Actual tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY']
Predicted tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY']
Test Example: 3
Sentence: sort the array route paths
Actual tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY', 'I-ARRAY']
Predicted tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY', 'I-ARRAY']
Test Example: 4
Sentence: sort the list enemy types
Actual tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY', 'I-ARRAY']
Predicted tags: ['B-OPERATION', 'O', 'O', 'O', 'B-ARRAY']
Test Example: 5
Sentence: sort the array parameters
Actual tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY']
Predicted tags: ['B-OPERATION', 'O', 'O', 'B-ARRAY']
Test 

In [177]:
# evaluation 
def evaluate(y_true, y_pred):
    mlb = MultiLabelBinarizer()
    y_true_binarized = mlb.fit_transform(y_true)
    y_pred_binarized = mlb.transform(y_pred)
    
    f1 = f1_score(y_true_binarized, y_pred_binarized, average='weighted')
    precision = precision_score(y_true_binarized, y_pred_binarized, average='weighted')
    recall = recall_score(y_true_binarized, y_pred_binarized, average='weighted')
    accuracy = accuracy_score(y_true_binarized, y_pred_binarized)
    
    return f1, precision, recall, accuracy

f1, precision, recall, accuracy = evaluate(labels, predicted_tags)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)

F1 Score: 0.9985534370803456
Precision: 1.0
Recall: 0.997254004576659
Accuracy: 0.9896013864818024


In [178]:
torch.save(ner_model.state_dict(), '../models/final_constrained_ner_model.pth')

In [179]:
ner_model.eval()
with torch.no_grad():
        for sentence, _, intent in test_data:
                precheck_sent = prepare_sequence(sentence.split(), word_to_index)
                # precheck_intent = prepare_sequence(intent, intent_to_index)
                precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
                mask = create_mask(intent, all_tags, intent_to_tags)
                predictions.append(ner_model(precheck_sent, precheck_intent, mask)[0])
    

Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi De